In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import warnings
import ipython_blocking

from numpy import linalg as LA
from IPython.core.display import Javascript, display
from ipywidgets import Button, Layout
from IPython.display import HTML

warnings.simplefilter('ignore')

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите на кнопку чтобы увидеть код"></form>''')

In [3]:
%%javascript
require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}

<IPython.core.display.Javascript object>

### Принцип минимакса в решении игровых задач

#### Основные понятия

&#8195;&#8195; Во многих практических задачах возникают ситуации, когда требуется принять решение, не имея достаточной информации. Неизвестными могут быть как условия осуществления какой-либо операции, так и сознательные действия лиц, от которых зависит успех этой операции.

&#8195;&#8195; Матричной игрой называется игра, осуществляемая по следующим правилам:
- В игре участвуют два игрока;
- Каждый из игроков обладает конечным набором стратегий;
- Игра заключается в том, что каждый из игроков, не имея информации о действиях противника, делает один ход (выбирает одну из своих стратегий). Результатом выбора игроками стратегий является выигрыш и проигрыш в игре.
- И выигрыш, и проигрыш выражаются числами.

#### Матричная игра с нулевой суммой

&#8195;&#8195; Матричная игра называется игрой с нулевой суммой, если в этой игре выигрыш одного игрока равняется проигрышу другого игрока.
        
&#8195;&#8195; Каждая матричная игра с нулевой суммой имеет платежную матрицу. Для того чтобы построить эту матрицу, обозначим одного из игроков символом A, а другого - символом B, и предположим, что $A_1$, $A_2$, ..., $A_m$ - стратегии, которые может применять игрок A, а $B_1$, $B_2$, ..., $B_n$ - стратегии, которые может применять игрок B.

&#8195;&#8195; Матричная игра, в которой у игрока A имеется m стратегий, а у игрока B - n стратегий, называется игрой типа m$\times$n.

#### Нижняя и верхняя цена игры. Принцип минимакса

Рассмотрим матричную игру типа m$\times$n с платежной матрицей 
        \begin{array}{cccc}
            c_{11} & c_{12} & \ldots & c_{1n}\\
            c_{21} & c_{22} & \ldots & c_{2n}\\
            \vdots & \vdots & \ddots & \vdots\\
            c_{m1} & c_{m2} & \ldots & c_{mn}
        \end{array}
        
&#8195;&#8195; Если игрок A выберет стратегию $A_i$, то все его возможные выигрыши будут элементами i-й строки матрицы C. В наихудшем для игрока A случае, когда игрок B применяет стратегию, соответствующую минимальному элементу этой строки, выигрыш игрока A будет равен числу min $c_{ij}$, 1$\leq$j$\leq$n.
    
&#8195;&#8195; Следовательно, для получения наибольшего выигрыша, игроку A нужно выбирать ту из стратегий, для которой число min $c_{ij}$ (1$\leq$j$\leq$n) максимально. Это значение принятно считать нижней ценой игры.

&#8195;&#8195;  Следовательно, игроку B нужно выбрать такую стратегию, для которой число max $c_{ij}$ (1$\leq$i$\leq$m) минимально. Это значение принято называть верхней ценой игры.

#### Игры с седловой точкой

Игра называется игрой с седловой точкой, если ее нижняя и верхняя цены совпадают, то есть выполняется равенство
    
&#8195;&#8195; $\alpha$=max min $c_{ij}$=min max $c_{ij}$=$\beta$, 1$\leq$i$\leq$m, 1$\leq$j$\leq$n
    
Для игры с седловой точкой общее значение нижней и верхней цены игры
    
&#8195;&#8195; V=$\alpha$=$\beta$

называется ценой игры. Если седловая точка существует, то игра заканчивается после нахождения цены игры.

&#8195;&#8195; Если селовая точка отсутствует, то в чистых стратегиях рассматриваемая игра не разрешима. Значит, результат этой игры определится в смешанных стратегиях.

#### Теперь перейдем к практическому решению матричной игры.

&#8195;&#8195; Для начала необходимо ввести матрицу, вы сможете сделать это ниже.

In [4]:
%%html
<style>
.enjoy-css {
  display: inline-block;
  -webkit-box-sizing: content-box;
  -moz-box-sizing: content-box;
  box-sizing: content-box;
  cursor: pointer;
  padding: 10px 20px;
  border: 1px solid rgba(0,0,0,0);
  -webkit-border-radius: 7px;
  border-radius: 7px;
  font: normal 12px/normal Verdana, Geneva, sans-serif;
  color: rgba(0,0,0,1);
  -o-text-overflow: clip;
  text-overflow: clip;
  word-spacing: 6px;
  background: rgba(255,255,0,0.34);
}
</style>
<div class="enjoy-css">После заполнения формы не изменяйте введенные данные!</div>

In [5]:
matrix_rows_counter = widgets.IntSlider(
    value=2,
    min=2,
    max=3,
    step=1,
    description='Количество строк в матрице:',
    style={"description_width": 'initial',
          "handle_color": 'lightblue'},
    layout=Layout(width='40%')
)

matrix_cols_counter = widgets.IntSlider(
    value=2,
    min=2,
    max=5,
    step=1,
    description='Количество cтолбов в матрице:',
    style={"description_width": 'initial',
          "handle_color": 'lightblue'},
    layout=Layout(width='40%')
)
display(matrix_rows_counter)
display(matrix_cols_counter)

IntSlider(value=2, description='Количество строк в матрице:', layout=Layout(width='40%'), max=3, min=2, style=…

IntSlider(value=2, description='Количество cтолбов в матрице:', layout=Layout(width='40%'), max=5, min=2, styl…

In [7]:
# Кнопка для приостановления выполнения следующих ячеек

block_button_1 = widgets.Button(description='Далее', layout=Layout(width='26.5%', height='30px'))
display(block_button_1)
%blockrun block_button_1

Button(description='Далее', layout=Layout(height='30px', width='26.5%'), style=ButtonStyle())

<IPython.core.display.Javascript object>

In [8]:
n_row = matrix_rows_counter.value
n_col = matrix_cols_counter.value

Значения для матричной игры:

In [20]:
def vbox_draw(rows,cols):
    
    vbox = widgets.VBox([widgets.Tab(title='') for i in range(rows)])

    #   Cоздание виджета для ограничений

    for i in range(len(vbox.children)):
        vbox.children[i].set_title(0,"")
    for i in range(rows):
        vbox.children[i].children = [widgets.IntText(description='') for i in range(cols)]
    display(vbox)
    return vbox
    
vbox = vbox_draw(n_row,n_col)

In [11]:
block_button_2 = widgets.Button(description='Выполнить расчет', layout=Layout(width='26.5%', height='30px'))
display(block_button_2)
%blockrun block_button_2

Button(description='Выполнить расчет', layout=Layout(height='30px', width='26.5%'), style=ButtonStyle())

<IPython.core.display.Javascript object>

In [12]:
def toFixed(f: float, n=0):
    a, b = str(f).split('.')
    return '{}.{}{}'.format(a, b[:n], '0'*(n-len(b)))

def init_data(matrix):
    
    for i in range(len(vbox.children)):
            for j in range(len(vbox.children[i].children)):
                matrix[i][j] = vbox.children[i].children[j].value
    return matrix


def invert_matrix(matrix):
    
    rows_cnt = len(matrix)
    cols_cnt = len(matrix[0])

    new_matix = [[0]*rows_cnt for _ in range(cols_cnt)]

    for i in range(rows_cnt):
        for j in range(cols_cnt):
            new_matix[j][i] = matrix[i][j]
    
    return new_matix


def delete_cols(mat,n):
    if len(mat[0]) > 3:
    
        columnsList = []
        columnsList1 = []
        check = 0

        for i in range(len(mat[0])):

            for j in range(i+1,len(mat[0])):

                if (len(columnsList)<6):

                    for k in range(len(mat)):

                        if (mat[k][i] >= mat[k][j]):
                            check = check + 1
                        else:
                            check = 0
                            break

                    if (check == len(mat)):
                        columnsList.append(i)
                        check = 0
                        break;

        check = 0               

        for i in range(len(mat[0])):

            for j in range(i+1,len(mat[0])):

                if (len(columnsList1)<6):

                    for k in range(len(mat)):

                        if (mat[k][j] >= mat[k][i]):
                            check = check + 1
                        else:
                            check = 0
                            break


                    if (check == len(mat)):
                        columnsList1.append(j)
                        check = 0
                        break;
        S = []

        if columnsList and columnsList1:
            temp = np.hstack((columnsList, columnsList1))
            temp = np.unique(temp)
            temp = np.resize(temp, (len(mat[0])-3))
            S = np.delete(mat, temp, 1)
            S = np.transpose(S, axes = None)
            A = ['A'+str(i+1) for i in range(n)]
            B = {'B'+str(i+1): S[i] for i in range(len(S))}

        else:
            if columnsList1:
                temp = np.resize(columnsList1, (len(mat[0])-3))
                S = np.delete(mat, temp, 1)
                S = np.transpose(S, axes=None)
                A = ['A'+str(i+1) for i in range(n)]
                B = {'B'+str(i+1): S[i] for i in range(len(S))}


            if columnsList:
                temp = np.resize(columnsList, (len(mat[0])-3))
                S = np.delete(mat, temp, 1)
                S = np.transpose(S, axes=None)
                A = ['A'+str(i+1) for i in range(n)]
                B = {'B'+str(i+1): S[i] for i in range(len(S))}
                
            if not columnsList and not columnsList1:
                return invert_matrix(mat)
            
        return S
    return mat

def game2x2(a):
    
    p1 = (a[1][1] - a[1][0])/(a[0][0]+a[1][1] - a[1][0] - a[0][1])
    p2 = (a[0][0] - a[0][1])/(a[0][0]+a[1][1] - a[1][0] - a[0][1])
    
    q1 = (a[1][1] - a[0][1])/(a[0][0]+a[1][1] - a[1][0] - a[0][1])
    q2 = (a[0][0] - a[1][0])/(a[0][0]+a[1][1] - a[1][0] - a[0][1])
    
    y = (a[0][0]*a[1][1] - a[0][1]*a[1][0])/(a[0][0]+a[1][1] - a[1][0] - a[0][1])
    
    print("Ответ: y = {}".format(y))
    print("Смешанные стратегии игрока №1: P({},{}".format(p1,p2))
    print("Смешанные стратегии игрока №2: Q({},{}".format(q1,q2))
    
def find_first_p(temp):

    right_side = np.array([[0], [0], [0],[1]])
    bottom_add = np.array([1,1,1])
    minus_y = np.array([[-1],[-1],[-1],[0]])

    bottom_add = bottom_add.reshape(1,3)
    minus_y = minus_y.reshape(4,1)
    right_side = right_side.reshape(4,1)
    
    temp_matrix = np.concatenate([temp, bottom_add], axis = 0)
    temp_matrix = np.concatenate([temp_matrix, minus_y], axis = 1)

    x = LA.solve(temp_matrix, right_side)
    return x

def find_second_p(temp):

    right_side = np.array([[0], [0], [0],[1]])
    bottom_add = np.array([1,1,1])
    minus_y = np.array([[-1],[-1],[-1],[0]])
    
    bottom_add = bottom_add.reshape(1,3)
    minus_y = minus_y.reshape(4,1)
    right_side = right_side.reshape(4,1)

    temp_matrix1 = np.transpose(temp, axes = None)
    temp_matrix1 = np.concatenate([temp_matrix1, bottom_add], axis = 0)
    temp_matrix1 = np.concatenate([temp_matrix1, minus_y], axis = 1)

    x1 = LA.solve(temp_matrix1, right_side)
    return x1

def extremum_counts(matrix, inv_m, df, A, styles):
    
    amins = []
    bmaxs = []
        
    for i in range(1,len(A)+1):
        amins.append(df.loc["A"+str(i)].min())

    for i in range(1,len(inv_m)+1):
        bmaxs.append(df["B"+str(i)].max())

    df['a=min'] = amins
    df.loc['b=max'] = bmaxs + [np.nan]
    
    html = (df.style.set_table_styles(styles))
    display(html)
        
    return {'dataframe':df, 'amins':amins, 'bmaxs': bmaxs}

#### Шаг №1 Получение платежной матрицы.

In [13]:
pd.set_option('precision', 0)
df_matrix = init_data(np.zeros((n_row,n_col)))
invert_m = invert_matrix(df_matrix)                             # Для корректного отображения данных в датафрейме
A = ['A'+str(i+1) for i in range(n_row)]                        # Индексы для строк df
B = {'B'+str(i+1): invert_m[i] for i in range(len(invert_m))}   # Индексы для столбцов df
df = pd.DataFrame(index=A,data=B)

In [14]:
# Стилизация датафрейма

def hover(hover_color="#ffff99"):
    
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    dict(selector="th", props=[("font-size", "120%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom")]),
    dict(selector="tr", props=[("font-size", "150%"),
                               ("text-align", "center")])
]
html = (df.style.set_table_styles(styles))
display(html)

,B1,B2,B3,B4,B5
A1,3,-4,7,-3,-5
A2,5,5,-2,5,-10
A3,-4,-5,9,11,-2


#### Шаг №2 Расчет максимальных и минимальных значений.

&#8195;&#8195; На данном этапе необходимо рассчитать минимальные значения для каждой строки в матрице, а также максимальные значения для каждого столбца в матрице.

#### Но если матрица в введенном вами примере имеет размерность 3xN, где N > 3, то перед вычислением минимальных и максимальных значений по строкам и столбцам необходимо убрать доминирующие столбцы.

После отсеивания доминирующих столбцов, у нас получилась следующая матрица:

In [15]:
temp = delete_cols(df_matrix,n_row)
A = ['A'+str(i+1) for i in range(n_row)]
B = {'B'+str(i+1): temp[i] for i in range(len(temp))}
df = pd.DataFrame(index=A,data=B)
html = (df.style.set_table_styles(styles))
display(html)

,B1,B2,B3
A1,-4,-3,-5
A2,5,5,-10
A3,-5,11,-2


Теперь требуется выполнить те же действия, что были указаны в начале второго шага. А именно рассчитать минимальные значения для каждой строки в матрице, а также максимальные значения для каждого столбца в матрице.

In [16]:
extr_count = extremum_counts(df_matrix,temp,df,A,styles)

,B1,B2,B3,a=min
A1,-4,-3,-5,-5
A2,5,5,-10,-10
A3,-5,11,-2,-5
b=max,5,11,-2,nan


#### Шаг №3 Поиск нижней и верхней цены игры

Для нахождения верхней  нижней цены игры требуется:
- Для нижней цены игры необходимо найти максимум среди минимальных в строках;
- Для верхней цены игры необходимо найти минимум среди максимумов по столбцам.

Седловая точка будет существовать только в том случае, если нижняя и верхняя границы равны.

In [17]:
# проверка на седловую точку

sed_point = False

max_point = max(extr_count['amins'])
min_point = min(extr_count['bmaxs'])

print("Нижняя цена игры: {}".format(max_point))
print("Верхняя цена игры: {}".format(min_point))

if (max_point == min_point):
    print("Седловая точка существует, игра закончится в чистых стратегиях")
    print("Цена игры: {}".format(max_point))
    sed_point = True
else:
    print("Cедловой точки не существует, игра решится в смешанных стратегиях")
    if len(temp) != n_row:
        print('Данная игра не решается стандартными методами, необходимо использовать симплекс-метод')

Нижняя цена игры: -5.0
Верхняя цена игры: -2.0
Cедловой точки не существует, игра решится в смешанных стратегиях


#### Если решения игры в чистых стратегиях не существует, то применяются смешанные. О них вы также сможете прочитать в лекционных материалах.

&#8195;&#8195; Для того чтобы найти решение в смешанных стратегиях, необходимо составить систему линейных неравенств из получившейся на данном этапе матрицы. Когда система будет решена, вы сможете найсти стратегии каждого игрока, а также цену игры.

In [18]:
if (sed_point == False and n_row == 2 and len(temp) == 2):
    game2x2(df_matrix)

In [19]:
if (sed_point == False and n_row == 3 and len(temp) == 3):
    marker1 = True
    marker2 = True

    temp = delete_cols(df_matrix, n_row)
    player_1 = find_first_p(temp)
    player_1 = np.resize(player_1, (1, 4))
    player_1 = player_1[0]

    player_2 = find_second_p(temp)
    player_2 = np.resize(player_1, (1, 4))
    player_2 = player_2[0]

    for i in range(3):
        if player_1[i] < 0:
            marker1 = False
        if player_2[i] < 0:
            marker2 = False

    if not marker1 or  not marker2:
        print("Вероятность стратегии отрицательная, решение данным способом получить невозможно")

    else:

        print("y = "+ str(toFixed(player_1[3], 1)))
        print("Смешанные стратегии игрока №1: P({}, {}, {})".format(player_1[0], player_1[1],player_1[2]))
        print("Смешанные стратегии игрока №2: Q({}, {}, {})".format(player_2[0], player_2[1],player_2[2]))

Вероятность стратегии отрицательная, решение данным способом получить невозможно
